## 1. 라이브러리 임포트

In [1]:
# 라이브러리 임포트
import pandas as pd   # 판다스
import altair as alt  # 알테어

## 2. 데이터 적재

In [2]:
# cctv 및 인구 데이터 적재 및 확인
df = pd.read_csv('https://raw.githubusercontent.com/logistex/vd21/main/cctv_with_pop.csv')
df.sample(5)

,no,id,자치구,CCTV_규모_2015,CCTV_규모_2020,인구_규모_2015,인구_규모_2020
1,11,2,중구,654,2001,134329,134635
2,24,3,용산구,685,2383,247909,244645
23,19,24,송파구,520,2743,667480,673926
20,15,21,관악구,2004,4909,529031,509803
14,22,15,양천구,2241,3307,489010,457781


In [3]:
# 지도 시각화를 위한 자치구 위치 데이터 적재 및 확인
offices = pd.read_csv('https://raw.githubusercontent.com/logistex/vd21/main/offices.csv')
offices.sample(5)

,자치구,자치구청,위도,경도,주소
19,동작구,동작구청,37.512430,126.939800,대한민국 서울특별시 동작구 노량진2동 장승배기로 161
10,노원구,노원구청,37.654047,127.056337,대한민국 서울특별시 노원구 상계6.7동 노해로 437
12,서대문구,서대문구청,37.579173,126.936781,대한민국 서울특별시 서대문구 연희동 연희로 248
16,구로구,구로구청,37.495470,126.887639,대한민국 서울특별시 구로구 구로동 가마산로 245
4,광진구,광진구청,37.538533,127.082377,대한민국 서울특별시 광진구 자양1동 자양로 117


In [4]:
# 자치구 경계선에 관한 지도 데이터
seoul_url = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/juso/2015/json/seoul_municipalities_topo.json'
seoul = alt.topo_feature(seoul_url, 'seoul_municipalities_geo')
seoul

UrlData({
  format: TopoDataFormat({
    feature: 'seoul_municipalities_geo',
    type: 'topojson'
  }),
  url: 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/juso/2015/json/seoul_municipalities_topo.json'
})

## 3. 버블차트 시각화

### 3.1 회귀선을 포함한 버블차트를 반환하는 함수 정의 

- 함수 `bubble(year, color)`를 정의하라. 
- 매개변수 `year`에 정수 `2015`를 지정하면 2015년 차트를 반환한다. 
- 매개변수 `color`에 색상 이름을 지정하면, 해당 색상으로 버블과 회귀선이 그려진 차트를 반환한다. 


In [5]:
# 회귀선 버블차트 시각화 함수 정의

def bubble(year, color):
    bb = alt.Chart(df).mark_circle(color=color, opacity=0.5).transform_calculate(
      CCTV_비율='datum.CCTV_규모_' + year + '/ datum.인구_규모_' + year
    ).encode(
        alt.X('인구_규모_' + year + ':Q',
              scale=alt.Scale(zero=False),
              title='인구 규모 (단위: 명)'),
        alt.Y('CCTV_규모_' + year + ':Q',
              scale=alt.Scale(domain=(0, 7000)),
              title='CCTV 설치 규모(단위: 대)'),
        alt.Size('CCTV_비율:Q', scale=alt.Scale(range=[1, 1000]), legend=None),
        tooltip = [
                   '자치구:N',
                   alt.Tooltip('CCTV_비율:N', format='.1%', title='CCTV 비율'),
                   alt.Tooltip('CCTV_규모_' + year + ':Q', title='CCTV 규모'),
                   alt.Tooltip('인구_규모_' + year + ':Q', title='인구 규모')
        ]
    ).properties(
        width=600,
        height=500
    )


    reg = bb.transform_regression('인구_규모_' + year, 'CCTV_규모_' + year).mark_line(color=color, opacity=0.5)

    return alt.layer(bb, reg)   # 버블 차트 위에 회귀선을 겹쳐서 표시         

### 3.2 2015년도 차트 시각화

In [6]:
# 2015년도 차트를 파란색으로 시각화
bb_reg_2015 = bubble('2015','blue')
bb_reg_2015

alt.LayerChart(...)

### 3.3 2020년도 차트 시각화

In [7]:
# 2020년도 차트를 빨간색으로 시각화
bb_reg_2020 = bubble('2020', 'red')
bb_reg_2020

alt.LayerChart(...)

### 3.4 시각화 결과 통합

In [8]:
# 시각화 결과 통합

layer = alt.layer(bb_reg_2015, bb_reg_2020)
layer

alt.LayerChart(...)

## 4. 버블 차트 시각화 결론

1. 회귀선보다 위에 있는 자치구는 상대적으로 CCTV가 많이 설치된 자치구이고, 반대로 아래에 있는 자치구는 상대적으로 CCTV가 적게 설치된 자치구입니다.

2. 원 크기를 통해 2015년도 에서는 종로구, 2020년도 에서는 중구가 인구 대비 CCTV설치 규모가 가장 크다는 것을 알 수 있습니다. 
 
3. 동그라미 분포도를 보면 자치구마다 표준편차가 커지는 것을 확인할 수 있는데, 이는 2020년도 인구가 늘어날수록 평균적인 CCTV 개수가 늘어나고 있음을 알 수 있습니다. 